In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
import os, sys, json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm as tqdm

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
sys.path.append('../model_options')
from feature_extraction import *


ModuleNotFoundError: No module named 'feature_extraction'

In [14]:
!pip install .

Processing /Users/garyk/Documents/code/neuro/DeepDive
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for deepdive: filename=deepdive-0.1.0-py3-none-any.whl size=1146 sha256=4becc0fd27181920b592eaf6a46f0e19b91434df4b75e3b316ea33cdea7beebf
  Stored in directory: /private/var/folders/t8/z05p65mn3j997gd006knx_lc0000gn/T/pip-ephem-wheel-cache-8bjybzap/wheels/c3/5b/fa/9541d88274192506aebe5f5e3d59d52a95ec6079d663e0c5c7
  Created wheel for visualpriors: filename=visualpriors-0.3.5-py3-none-any.whl size=11372 sha256=386cfad39f10f2b5e499a008a02b0da0df115c6cd35d39c07bc7ce5c0ebc37a3
  Stored in directory: /Users/garyk/Library/Caches/pip/wh

In [15]:
from DeepDive.feature_extraction import *
from DeepDive.model_options import *

### Model Metadata Build

In [16]:
def get_model_metadata(model_option, convert_to_dataframe = True):
    model_name = model_option['model_name']
    train_type = model_option['train_type']
    model = eval(model_option['call'])
    
    layer_metadata = get_feature_map_metadata(model)
    layer_count = len(layer_metadata.keys())
    layer_order = [model_layer for model_layer in layer_metadata]
    feature_counts = [layer_metadata[layer]['feature_count'] 
                      for layer in layer_metadata]
    parameter_counts = [layer_metadata[layer]['parameter_count'] 
                        for layer in layer_metadata]
    feature_map_shapes = [layer_metadata[layer]['feature_map_shape'] 
                          for layer in layer_metadata]
    total_feature_count = int(np.array(feature_counts).sum())
    total_parameter_count = int(np.array(parameter_counts).sum())
    model_metadata = {'total_feature_count': total_feature_count,
                      'total_parameter_count': total_parameter_count,
                      'layer_count': layer_count,
                      'layer_metadata': layer_metadata}
    
    if not convert_to_dataframe:
        return(model_metadata)
        
    if convert_to_dataframe:

        model_metadata_dictlist = []
        
        for layer_index, layer in enumerate(layer_metadata):
            model_metadata_dictlist.append({'model': model_name, 'train_type': train_type,
                                            'model_layer': layer, 'model_layer_index': layer_index + 1,
                                            'model_layer_depth': (layer_index + 1) / layer_count,
                                            'feature_count': layer_metadata[layer]['feature_count'],
                                            'parameter_count': layer_metadata[layer]['parameter_count']})

        return(pd.DataFrame(model_metadata_dictlist))

In [21]:
model_options = {**get_model_options(model_type='imagenet'), 
                 **get_model_options(train_type='taskonomy')}

model_metadata_dflist = []

def process(model_option):
    incoming_metadata = get_model_metadata(model_options[model_option])
    model_metadata_dflist.append(incoming_metadata)
    
problematic_model_options = []

def remark(model_option):
    problematic_model_options.append(model_option)
    
model_option_iterator = tqdm(model_options)
for model_option in model_option_iterator:
    model_option_iterator.set_description(model_option)
    try: process(model_option)
    except: remark(model_option)

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [9]:
pd.concat(model_metadata_dflist).to_csv('model_metadata.csv', index = None)

if 'model_metadata_json' in globals():
    with open('model_metadata.json', 'w') as file:
        json.dump(model_metadata_json, file)

In [ ]:
model_performance_data = pd.read_csv('model_scores.csv')#.set_index('model').to_dict('index')